In [9]:
%load_ext autoreload
%autoreload 2

import time

import ipyvuetify as v
import xlwings as xw
import ipywidgets as w
import pandas as pd

from interface import interface

#open excel
wb=xw.Book("IMS.xlsm")


#Instruments
type_of_instruments = ['Equity', 'Bond', 'Fund']
buy_sell = ["Buy","Sell"]


instruments = v.Select(v_model='Equity',id="instrument_select",
    label='Instrument',
    items=type_of_instruments)

def select_change(widget, event, data):
    content_list = [v.TabItem(children=[forms[(instruments.v_model,'Buy')]]),v.TabItem(children=[forms[(instruments.v_model,'Sell')]])]
    tabs = v.Tabs(
    v_model=1, 
    children=tab_list + content_list, color ="blue-grey darken-1")
    content.children = [instruments,tabs]

instruments.on_event('change',select_change)

widgets = {}
cells = {}
from copy import deepcopy
for item in interface:
    item = deepcopy(item)
    instrument, type_, obj = item.pop('INSTRUMENT'), item.pop('TYPE'), item.pop('OBJECT')
    widgets.setdefault((instrument, type_), [])
    cells.setdefault((instrument, type_), [])
    widget = getattr(v, obj)(**item)
    widget.id = f"{instrument}_{type_}_{item['label'].replace(' ','_')}"
    widgets[instrument, type_].append(widget)
    cells[instrument, type_].append((item['label'],item['cell']))


#forms    
forms = {}
for k, children in widgets.items():
        forms[k] = v.Form(children=children)

class AllForms:
    def __init__(self):
        for k, form in forms.items():
            for widget in form.children:
                setattr(self, widget.id, widget)

af = AllForms()

#Buttons Form
Proceed = v.Btn(
                  icon = False,
                  children=[
                      v.Icon(children=['mdi-application-export'],  class_ = "mr-2"),
                      'Proceed',
                      ])

def compile_excel(form,sheet_sell,cells):
    for f in cells:
        sheet_sell.v_model= sheet_sell.range(f)

#functions to be called
def Test(sheet_sell):
    sheet_sell.range("C19").value= ticker.v_model  
    if sheet_sell.range("C27").value == "#N/A Requesting Data..." or sheet_sell.range("C30").value == "#N/A Requesting Data...":
        time.sleep(0.1)
        Test(sheet_sell)
    else:
        #print(sheet_sell.range("C27").value)
        Px_Last.v_model = sheet_sell.range("C27").value
        ISIN.v_model = sheet_sell.range("B20").value
        Name.v_model = sheet_sell.range("C20").value
        Crncy.v_model = sheet_sell.range("C28").value
        Cross.v_model = sheet_sell.range("C29").value
        App_Cross.v_model = sheet_sell.range("C30").value
        Price_Euro.v_model = sheet_sell.range("C31").value
        #print(st.range("A5").value)

    #ticker.v_model = sheet_sell.range("C25").value

    
def verify(widget, event, data):
    Proceed.disabled = False

Proceed.on_event('click',Test)




    
#form= v.Form(children=[ticker,widgets.HBox([ISIN, Name],style_='width: 100%; opacity: 0.7'),
#                       Px_Last,Crncy,Cross,App_Cross,Price_Euro,Quantity,Verify, Proceed])


#tabs
content_list = [v.TabItem(children=[forms[('Equity','Buy')]]),v.TabItem(children=[forms[('Equity','Sell')]])]
tab_list = [v.Tab(children=["Pre-Trade Buy"]), v.Tab(children=["Pre-Check Sell"])]

tabs = v.Tabs(
    v_model=1, 
    children=tab_list + content_list, color ="blue-grey darken-1")

###############
#APP
###############

#navigation bar
navDrawer = v.NavigationDrawer(v_model=True, children=[
     v.Select(label='select1', items=[1,2,], prepend_icon = 'fa-truck'), 
    v.TextField(label='text', prepend_icon = 'mdi-heart'),selec
    v.Select(label='select3', prepend_icon = 'mdi-magnify')
])

navDrawer.mini_variant = True
navDrawer.expand_on_hover = True

#navDrawer.width =300
navDrawer.mini_variant_width = 30
navDrawer.mini_variant_width = 0

toolBarButton = v.Btn(
                  icon = True, 
                  children=[
                      v.Icon(children=['mdi-dots-vertical'])
                      ])

def on_click(widget, event, data):
    navDrawer.v_model = not navDrawer.v_model
    if navDrawer.v_model:
        navDrawer.mini_variant_width = 30
    else:
        navDrawer.mini_variant_width = 0
        
#toolBarButton.on_event('click', on_click)

#Init
v.theme.dark = True
app = v.App(v_model=1)

appBar = v.AppBar(
      #color="light-blue darken-3",
    color="blue-grey darken-1",
      dense=True,
      dark = True   ,  
       children = [
           toolBarButton, 
           v.ToolbarTitle(children=['Niche IMS']),
                    ])

content = v.Col(children = [instruments,tabs,Proceed])


drawersWithContent = v.Container(
        class_="fill-height",
         fluid=True,
        children = [
                     v.Row(
                          align="top",
                      justify="left",
                        children = [navDrawer,content]
                            )
                ])

app.children = [appBar,drawersWithContent]


app

SyntaxError: invalid syntax (3019015602.py, line 126)

In [3]:
%load_ext autoreload
%autoreload 2

import time

import ipyvuetify as v
import xlwings as xw
import ipywidgets as w
import pandas as pd

from interface import interface

#open excel
wb=xw.Book("IMS.xlsm")


#Instruments
type_of_instruments = ['Equity', 'Bond', 'Fund']
buy_sell = ["Buy","Sell"]


instruments = v.Select(v_model='Equity',id="instrument_select",
    label='Instrument',
    items=type_of_instruments)

instruments

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Select(id='instrument_select', items=['Equity', 'Bond', 'Fund'], label='Instrument', v_model='Equity')

In [ ]:
cells

NameError: name 'cells' is not defined

In [53]:
af

In [54]:


instruments = v.Select(v_model='Equity',id="instrument_select",
    label='Instrument',
    items=type_of_instruments)

def select_change(widget, event, data):
    content_list = [v.TabItem(children=[forms[(instruments.v_model,'Buy')]]),v.TabItem(children=[forms[(instruments.v_model,'Sell')]])]
    tabs = v.Tabs(
    v_model=1, 
    children=tab_list + content_list, color ="blue-grey darken-1")
    content.children = [instruments,tabs]


instruments.on_event('change',select_change)


['Bond_Buy_Accrued',
 'Bond_Buy_App.cross',
 'Bond_Buy_Broker',
 'Bond_Buy_Cash-out_euro',
 'Bond_Buy_Cross',
 'Bond_Buy_Currency',
 'Bond_Buy_Days_Acc',
 'Bond_Buy_ISIN',
 'Bond_Buy_Interest_Accr.',
 'Bond_Buy_Market_Value_Local_Currency',
 'Bond_Buy_Mininum_Piece',
 'Bond_Buy_Px_Last',
 'Bond_Buy_Quantity',
 'Bond_Buy_Trade_as_a_%_NAV',
 'Bond_Sell_%_of_the_position_to_be_sold',
 'Bond_Sell_Accrued',
 'Bond_Sell_App.cross',
 'Bond_Sell_Broker',
 'Bond_Sell_Cash-in_euro',
 'Bond_Sell_Cross',
 'Bond_Sell_Currency',
 'Bond_Sell_Days_Acc',
 'Bond_Sell_ISIN',
 'Bond_Sell_Interest_Accr.',
 'Bond_Sell_Mininum_Piece',
 'Bond_Sell_Px_Last',
 'Bond_Sell_Quantity',
 'Bond_Sell_Total_existing_position',
 'Bond_Sell_Trade_as_a_%_NAV',
 'Equity_Buy_App.cross',
 'Equity_Buy_Broker',
 'Equity_Buy_Cash-out_euro',
 'Equity_Buy_Cross',
 'Equity_Buy_Currency',
 'Equity_Buy_ISIN',
 'Equity_Buy_Name',
 'Equity_Buy_Price_Euro',
 'Equity_Buy_Px_Last',
 'Equity_Buy_Quantity',
 'Equity_Buy_Ticker_Bloomberg',


In [55]:
forms

{('Equity',
  'Buy'): Form(children=[TextField(disabled=False, filled=False, id='Equity_Buy_Ticker_Bloomberg', label='Ticker Bloomberg', v_model=None), TextField(disabled=True, filled=True, id='Equity_Buy_ISIN', label='ISIN', v_model=None), TextField(disabled=True, filled=True, id='Equity_Buy_Name', label='Name', v_model=None), TextField(disabled=True, filled=True, id='Equity_Buy_Px_Last', label='Px Last', v_model=None), TextField(disabled=True, filled=True, id='Equity_Buy_Currency', label='Currency', v_model=None), TextField(disabled=True, filled=True, id='Equity_Buy_Cross', label='Cross', v_model=None), TextField(disabled=True, filled=True, id='Equity_Buy_App.cross', label='App.cross', v_model=None), TextField(disabled=True, filled=True, id='Equity_Buy_Price_Euro', label='Price Euro', v_model=None), TextField(disabled=False, filled=False, id='Equity_Buy_Quantity', label='Quantity', v_model=0.0), TextField(disabled=True, filled=True, id='Equity_Buy_Cash-out_euro', label='Cash-out euro

In [94]:
forms

{('Equity',
  'Buy'): Form(children=[Select(id='Equity_Buy_To_work_%_volume', items=['1/3', '1/5', '1/7'], label='To work % volume', v_model=None)]),
 ('Fund',
  'Buy'): Form(children=[Select(id='Fund_Buy_Broker', items=['Not specified', 'Equita Sim', 'DBS Vickers', 'Samsung Securities', 'GMSA', 'Tavira Securities', 'BOFAMLY', 'IMI'], label='Broker', v_model=None)]),
 ('Bond',
  'Buy'): Form(children=[TextField(disabled=True, filled=True, id='Bond_Buy_Trade_as_a_%_NAV', label='Trade as a % NAV', v_model=None)]),
 ('Equity',
  'Sell'): Form(children=[Select(id='Equity_Sell_To_work_%_volume', items=['1/3', '1/5', '1/7'], label='To work % volume', v_model='1/5')]),
 ('Fund',
  'Sell'): Form(children=[Select(id='Fund_Sell_Broker', items=['Not specified', 'Equita Sim', 'DBS Vickers', 'Samsung Securities', 'GMSA', 'Tavira Securities', 'BOFAMLY', 'IMI'], label='Broker', v_model=None)]),
 ('Bond',
  'Sell'): Form(children=[TextField(disabled=True, filled=True, id='Bond_Sell_Trade_as_a_%_NAV', 

In [13]:
%load_ext autoreload
%autoreload 2

import time

import ipyvuetify as v
import xlwings as xw
import ipywidgets as w
import pandas as pd

from interface import interface

#open excel
wb=xw.Book("IMS.xlsm")

#Instruments
type_of_instruments = ['Equity', 'Bond', 'Fund']
buy_sell = ["Buy","Sell"]

instruments = v.Select(v_model='Equity',id="instrument_select",
    label='Instrument',
    items=type_of_instruments)

def select_change(widget, event, data):
    content_list = [v.TabItem(children=[forms[(instruments.v_model,'Buy')]]),v.TabItem(children=[forms[(instruments.v_model,'Sell')]])]
    tabs = v.Tabs(
    v_model=1, 
    children=tab_list + content_list, color ="blue-grey darken-1")
    content.children = [instruments,tabs]


instruments.on_event('change',select_change)

widgets = {}
cells = {}
from copy import deepcopy
for item in interface:
    item = deepcopy(item)
    instrument, type_, obj = item.pop('INSTRUMENT'), item.pop('TYPE'), item.pop('OBJECT')
    widgets.setdefault((instrument, type_), [])
    cells.setdefault((instrument, type_), [])
    widget = getattr(v, obj)(**item)
    widget.id = f"{instrument}_{type_}_{item['label'].replace(' ','_')}"
    widgets[instrument, type_].append(widget)
    cells[instrument, type_].append((item['label'],item['cell']))


#forms    
forms = {}
for k, children in widgets.items():
        forms[k] = v.Form(children=children)

class AllForms:
    def __init__(self):
        for k, form in forms.items():
            for widget in form.children:
                setattr(self, widget.id, widget)

af = AllForms()



#Buttons Form
Proceed = v.Btn(
                  icon = False,
                  children=[
                      v.Icon(children=['mdi-application-export'],  class_ = "mr-2"),
                      'Proceed',
                      ])

def compile_excel(form,sheet_sell,cells):
    for f in cells:
        sheet_sell.v_model= sheet_sell.range(f)

#functions to be called
def Test(sheet_sell):
    sheet_sell.range("C19").value= ticker.v_model  
    if sheet_sell.range("C27").value == "#N/A Requesting Data..." or sheet_sell.range("C30").value == "#N/A Requesting Data...":
        time.sleep(0.1)
        Test(sheet_sell)
    else:
        #print(sheet_sell.range("C27").value)
        Px_Last.v_model = sheet_sell.range("C27").value
        ISIN.v_model = sheet_sell.range("B20").value
        Name.v_model = sheet_sell.range("C20").value
        Crncy.v_model = sheet_sell.range("C28").value
        Cross.v_model = sheet_sell.range("C29").value
        App_Cross.v_model = sheet_sell.range("C30").value
        Price_Euro.v_model = sheet_sell.range("C31").value
        #print(st.range("A5").value)

    #ticker.v_model = sheet_sell.range("C25").value

    
def verify(widget, event, data):
    Proceed.disabled = False

Proceed.on_event('click',Test)




    
#form= v.Form(children=[ticker,widgets.HBox([ISIN, Name],style_='width: 100%; opacity: 0.7'),
#                       Px_Last,Crncy,Cross,App_Cross,Price_Euro,Quantity,Verify, Proceed])


#tabs
content_list = [v.TabItem(children=[forms[('Equity','Buy')]]),v.TabItem(children=[forms[('Equity','Sell')]])]
tab_list = [v.Tab(children=["Pre-Trade Buy"]), v.Tab(children=["Pre-Check Sell"])]

tabs = v.Tabs(
    v_model=1, 
    children=tab_list + content_list, color ="blue-grey darken-1")

###############
#APP
###############

#navigation bar
navDrawer = v.NavigationDrawer(v_model=True, children=[
     v.Select(label='select1', items=[1,2,], prepend_icon = 'fa-truck'), 
    v.TextField(label='text', prepend_icon = 'mdi-heart'),
    v.Select(label='select3', prepend_icon = 'mdi-magnify')
])

navDrawer.mini_variant = True
navDrawer.expand_on_hover = True

#navDrawer.width =300
navDrawer.mini_variant_width = 30
navDrawer.mini_variant_width = 0

toolBarButton = v.Btn(
                  icon = True, 
                  children=[
                      v.Icon(children=['mdi-dots-vertical'])
                      ])

def on_click(widget, event, data):
    navDrawer.v_model = not navDrawer.v_model
    if navDrawer.v_model:
        navDrawer.mini_variant_width = 30
    else:
        navDrawer.mini_variant_width = 0
        
#toolBarButton.on_event('click', on_click)

#Init
v.theme.dark = True
app = v.App(v_model=1)

appBar = v.AppBar(
      #color="light-blue darken-3",
    color="blue-grey darken-1",
      dense=True,
      dark = True   ,  
       children = [
           toolBarButton, 
           v.ToolbarTitle(children=['Niche IMS']),
                    ])

content = v.Col(children = [instruments,tabs,Proceed])


drawersWithContent = v.Container(
        class_="fill-height",
         fluid=True,
        children = [
                     v.Row(
                          align="top",
                      justify="left",
                        children = [navDrawer,content]
                            )
                ])

app.children = [appBar,drawersWithContent]


app

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


App(children=[AppBar(children=[Btn(children=[Icon(children=['mdi-dots-vertical'])], icon=True), ToolbarTitle(c…

In [12]:
%load_ext autoreload
%autoreload 2

import time

import ipyvuetify as v
import xlwings as xw
import ipywidgets as w
import pandas as pd

from interface import interface

#open excel
wb=xw.Book("IMS.xlsm")

#Instruments
type_of_instruments = ['Equity', 'Bond', 'Fund']
buy_sell = ["Buy","Sell"]

instruments = v.Select(v_model='Equity',id="instrument_select",
    label='Instrument',
    items=type_of_instruments)

def select_change(widget, event, data):
    content_list = [v.TabItem(children=[forms[(instruments.v_model,'Buy')]]),v.TabItem(children=[forms[(instruments.v_model,'Sell')]])]
    tabs = v.Tabs(
    v_model=1, 
    children=tab_list + content_list, color ="blue-grey darken-1")
    content.children = [instruments,tabs]


instruments.on_event('change',select_change)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
